In [9]:
import numpy as np 
import scipy as sp

In [59]:
import numpy as np

get_skew = lambda a: np.array([[ 0, -a[2, 0], a[1, 0]], [a[2, 0], 0, -a[0, 0]], [-a[1, 0], a[0, 0], 0]])

def get_w_from_skew(skew):
    return np.array([[skew[2, 1]], [skew[0, 2]], [skew[1, 0]]])

get_linear_angular_velocity_from_twist = lambda twist: (twist[:3][:3], twist[3:][:3])

create_twist_from_linear_angular_velocity = lambda linear_velocity, angular_velocity: np.vstack((linear_velocity, angular_velocity))

# Define the rotation matrices for each axis

def get_x_rotation_matrix(theta, is_hybrid = False):
    
    if is_hybrid:
        return np.array([
            [1, 0, 0, 0],
            [0, np.cos(theta), -np.sin(theta), 0],
            [0, np.sin(theta), np.cos(theta), 0],
            [0, 0, 0, 1]
        ])
    else:
        return np.array([[1, 0, 0],
                        [0, np.cos(theta), -np.sin(theta)],
                        [0, np.sin(theta), np.cos(theta)]])
    
def get_y_rotation_matrix(theta, is_hybrid = False):
    if is_hybrid:
        return np.array([
        [np.cos(theta), 0, np.sin(theta), 0],
        [0, 1, 0, 0],
        [-np.sin(theta), 0, np.cos(theta), 0],
        [0, 0, 0, 1]
    ])
    else:
        return np.array([[np.cos(theta), 0, np.sin(theta)],
               [0, 1, 0],
               [-np.sin(theta), 0, np.cos(theta)]])
    
def get_z_rotation_matrix(theta, is_hybrid = False):
    if is_hybrid:
        return np.array([
            [np.cos(theta), -np.sin(theta), 0, 0],
            [np.sin(theta), np.cos(theta), 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
    else:      
        return np.array([[np.cos(theta), -np.sin(theta), 0],
                [np.sin(theta), np.cos(theta), 0],
                [0, 0, 1]])

# 5.1

[![image.png](https://i.postimg.cc/wvb8dyZ2/image.png)](https://postimg.cc/2bvXQ6zL)

In [47]:

w_0_1 = w01 = np.array([[-2.53], [-0.90], [1.64]])

final_time = t = 8.00

Both General case and Special case give the same answer, the only thing that is different is the rigour

### Special case based on the info in the question T_s_b[0] = $I$

In [50]:
# T_s_b[t] = e^([V_s] * t) * T_s_b[0] 

# T_s_b[0] = I so

# T_s_b[t] = e^([V_s] * t)

w_s_skew = get_skew(w01)
V_s_skwq = np.eye(4)
V_s_skwq[0:3, 0:3] = w_s_skew * t
V_s_skwq[3, 3] = 0

ans = sp.linalg.expm(V_s_skwq)[0:3, 0:3]
ans

array([[ 0.99972697, -0.02030428, -0.01156379],
       [ 0.02065957,  0.99929078,  0.03148196],
       [ 0.01091637, -0.03171226,  0.99943742]])

### General case makes no assumptions

[![image.png](https://i.postimg.cc/Gh4MfSbZ/image.png)](https://postimg.cc/TpXqL0L9)

In [51]:
T_s_b = [0 for i in range(10)] 

t_inc = final_time / 10

curr_t = 0

curr_t += t_inc

T_s_b[0] = np.eye(4)

for i in range(1, 10):
    curr_t += t_inc # it is now the end of the interval

    w_s_skew = get_skew(w_0_1)
    V_s_skwq = np.eye(4)
    
    V_s_skwq[0:3, 0:3] = w_s_skew * curr_t
    T_s_b[i] = sp.linalg.expm(V_s_skwq) @ T_s_b[0]

ans = T_s_b[-1][0:3, 0:3]
ans


array([[ 0.99972697, -0.02030428, -0.01156379],
       [ 0.02065957,  0.99929078,  0.03148196],
       [ 0.01091637, -0.03171226,  0.99943742]])

# 5.2

[![image.png](https://i.postimg.cc/DZy0gMtN/image.png)](https://postimg.cc/HJRTWBW0)

In [69]:
R_0_1_after_motion = R = np.array([[-0.20942517, 0.39494557, -0.89451612], 
                                   [-0.09943930, 0.90145748, 0.42129116], 
                                   [0.97275533, 0.17717903, -0.14951477]])

In [70]:
w_skew = sp.linalg.logm(R_0_1_after_motion)
w_skew

array([[-2.97065966e-09,  4.57465808e-01, -1.72782965e+00],
       [-4.57465812e-01,  1.05198118e-09,  2.25882626e-01],
       [ 1.72782966e+00, -2.25882622e-01,  4.57962595e-09]])

In [75]:
w_t = get_w_from_skew(w_skew)

w_t

# R_0_1_after_motion = e^([w]*t)

array([[-0.22588262],
       [-1.72782965],
       [-0.45746581]])

In [76]:
a = np.array([[-0.125380237784], [-0.959063112855], [-0.253924676891]]) * 1.80158076129

a

array([[-0.22588262],
       [-1.72782965],
       [-0.45746581]])

# 5.3

In [80]:
R_1in0 = np.array([[-0.51946552, -0.21121293, -0.82797625], 
                   [-0.23026763, -0.89852233, 0.37367693], 
                   [-0.82288055, 0.38476841, 0.41811586]]) 

R_2in0 = np.array([[-0.25366003, 0.24947997, -0.93456746], 
                   [-0.65832404, -0.75240807, -0.02217095], 
                   [-0.70870731, 0.60962434, 0.35509451]])

# 5.4

In [126]:
import numpy as np

R_0_1 = R01 = R01 = np.array([[-0.380464043808, -0.770372041511, 0.511638572653], 
                              [0.785590490188, 0.022681275867, 0.618330931986], 
                              [-0.487949478014, 0.637191083885, 0.596567539783]])

w_0 = w0 = np.array([[2.640000000000], [-0.950000000000], [-2.920000000000]])

final_time = t = 9.850000000000

p_1_1 = p1 = np.array([[2.740000000000], [-1.960000000000], [2.840000000000]])

In [128]:
# [[5.52683078305], [-3.71729696938], [0.379095390703]]

p_0_initial = R_0_1 @ p_1_1


w_skew = np.zeros((4, 4))

w_0_skew = get_skew(w_0)

linear_veloctiy_of_point = w_0_skew @ p_0_initial


linear_veloctiy_of_point

# w_skew[0:4, 3]



array([[12.13028074],
       [-3.2540048 ],
       [12.02576908]])

In [136]:
w_skew[0:3, 3] = [linear_veloctiy_of_point[0][0], linear_veloctiy_of_point[1][0], linear_veloctiy_of_point[2][0]]


# p_0_0_final = 

p_0_initial = np.array([p_0_initial[0], p_0_initial[1], p_0_initial[2], [1]])

ans = sp.linalg.expm(w_skew * final_time) @ p_0_initial
ans = R_0_1.T @ ans[0:3]
ans


array([[-125.6982735 ],
       [ -19.25582464],
       [ 114.81924405]])

# 5.5

[![image.png](https://i.postimg.cc/vBZqHsNX/image.png)](https://postimg.cc/t18t2LyV)

In [148]:
V_sbinb = np.array([[-7], [ 8], [-7], [ 7], [-7], [-4]])

In [149]:
# angular velocity of the body frame in the space frame
w_sbinb = V_sbinb[:3][:3]
w_sbinb


array([[-7],
       [ 8],
       [-7]])

In [150]:
# Translation velocity of the body frame in the space frame
v_sbinb = V_sbinb[3:][:3]
v_sbinb

array([[ 7],
       [-7],
       [-4]])

In [151]:
V_sbinb = np.array([[-5], [ 0], [-1], [-8], [-6], [ 1]])

In [152]:
# angular velocity of the body frame in the space frame
w_sbinb = V_sbinb[:3][:3]
w_sbinb


array([[-5],
       [ 0],
       [-1]])

In [153]:
# Translation velocity of the body frame in the space frame
v_sbinb = V_sbinb[3:][:3]
v_sbinb

array([[-8],
       [-6],
       [ 1]])

# 5.6

In [154]:
V = np.array([[ 3], [ 4], [-2], [-8], [ 4], [-9]])